In [ ]:
import threading
import time

START_NUMBER = 10 ** 6

def decorate_stop_generator(
    stop_word: str = "STOP",
    poll_delay_seconds: float = 0.01,
    input_function=input,
):
    def decorator(base_gen):
        def enhanced_gen(*args, **kwargs):
            stop_event = threading.Event()

            def wait_for_stop():
                while not stop_event.is_set():
                    user_input = input_function(
                        f"Enter {stop_word!r} to terminate... "
                    )
                    if user_input.strip().upper() == stop_word.upper():
                        stop_event.set()

            stop_thread = threading.Thread(
                target=wait_for_stop,
                daemon=True,
            )
            stop_thread.start()

            output = []
            gen = base_gen(*args, **kwargs)

            while not stop_event.is_set():
                output.append(next(gen))
                if poll_delay_seconds > 0:
                    time.sleep(poll_delay_seconds)

            return(output)

        return(enhanced_gen)

    return(decorator)

@decorate_stop_generator(stop_word = "STOP", poll_delay_seconds = 0.05)
def generate_primes(start_number):
    test_number = start_number - 1 #Start at -1 so the inner loop resets to start_number
    while True: #Usually best to avoid while True but it's needed for running for arbitrarily long until stopped
        is_prime = True
        test_number += 1
        for test_factor in range(2, int(test_number ** 0.5) + 1): #If no factors < sqrt(x) then x is prime.
            if test_number % test_factor == 0:
                is_prime = False
                break
        if is_prime:
            yield(test_number) #Yield, not return, so we can nest the generator with next()

print(generate_primes(START_NUMBER))